In [1]:
from sqlalchemy import create_engine
import pandas as pd

from sqlalchemy.engine import URL

connection_string = "DRIVER={SQL Server};SERVER=localhost;DATABASE=brewery;Truster_Connections=yes"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

# Views:

#### 1) Вывод каталога товаров, его количества на складе, и цены со скидкой (сложный: таблицы products, stock)

```sql
create view ProductsStockView as
	select product_name,
		product_type,
		prime_price,
		retail_price,
		discount_price, 
		amount 
	from products
		join stock on products.product_id=stock.product_id
GO
```

In [2]:
tableResult = pd.read_sql("SELECT * FROM ProductsStockView", engine)
tableResult

,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 7,Пиво,123456789121,67.0,92.46,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5


#### 2) Вывод всех заказов и их содержимого (сложный: таблицы orders, order_details, customers)

```sql
create view OrdersOrderDetailsCustomersView as
select orders.order_id,
	   customer_name,
	   order_date,
	   orders.order_status,
	   products.product_id,
	   product_name,
	   discount_price,
	   amount,
	   total_cost
		from orders 
			join order_details on orders.order_id=order_details.order_id 
			join products on order_details.product_id=products.product_id
			join customers on orders.customer_id=customers.customer_id
go
```


In [3]:
#id
tableResult = pd.read_sql("SELECT * FROM OrdersOrderDetailsCustomersView", engine)
tableResult

,order_id,customer_name,order_date,order_status,product_id,product_name,amount,price,cost
0,1,Красное&Белое,2022-05-23 14:25:10,pending,1,Балтика 7,1,83.21,83.21
1,1,Красное&Белое,2022-05-23 14:25:10,pending,2,Балтика 8,2,93.84,187.68
2,1,Красное&Белое,2022-05-23 14:25:10,pending,3,Балтика 9,4,76.18,304.72
3,2,Красное&Белое,2022-05-24 14:25:10,processing,2,Балтика 8,2,93.84,187.68
4,2,Красное&Белое,2022-05-24 14:25:10,processing,3,Балтика 9,3,76.18,228.54
5,2,Красное&Белое,2022-05-24 14:25:10,processing,4,Strongbow яблоко,4,78.66,314.64
6,3,РосАл,2022-05-25 14:25:10,pending,1,Балтика 7,5,83.21,416.05
7,3,РосАл,2022-05-25 14:25:10,pending,3,Балтика 9,6,76.18,457.08
8,3,РосАл,2022-05-25 14:25:10,pending,5,Strongbow груша,7,56.03,392.21
9,4,РосАл,2022-05-26 14:25:10,completed,2,Балтика 8,2,93.84,187.68


# Процедуры (все запросы)

## Вывод

---
### 1) Вывод базы товаров для сотрудника завода с количеством товара на складе и себестоимости продукта(сложный)
```sql
create procedure ShowProductsBase 
as
select * from ProductsStockView
```

In [4]:
tableResult = pd.read_sql("exec ShowProductsBase", engine)
tableResult

,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 7,Пиво,123456789121,67.0,92.46,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5


---
### 2) Вывод каталога товаров для заказчика
```sql
create procedure ShowProductsCatalog 
as
select product_name,
	   product_type,
	   retail_price, 
	   discount_price 
from ProductsStockView
```

In [5]:
tableResult = pd.read_sql("""exec ShowProductsCatalog""", engine)
tableResult

,product_name,product_type,retail_price,discount_price
0,Балтика 7,Пиво,92.46,83.21
1,Балтика 8,Пиво,93.84,93.84
2,Балтика 9,Пиво,95.22,76.18
3,Strongbow яблоко,Сидр,78.66,78.66
4,Strongbow груша,Сидр,80.04,56.03


---
### 3) Получить информацию о недостающих товарах на складе для существующих заказов(сложный)
```sql
create procedure ShowLackProducts
as
select product_name, 
	   abs(amount_on_stock - ordered) as lack_on_stock 
from (select product_name, 
		sum(OrdersOrderDetailsCustomersView.amount) as ordered, 
		stock.amount as amount_on_stock
	from OrdersOrderDetailsCustomersView 
		join stock 
		  on OrdersOrderDetailsCustomersView.product_id=stock.product_id
	  where order_status != 'completed'
	  group by product_name, stock.amount) 
as a
where (amount_on_stock - ordered) < 0
```

In [6]:
tableResult = pd.read_sql("""exec ShowLackProducts""", engine)
tableResult

,product_name,lack_on_stock
0,Балтика 7,11
1,Балтика 8,2
2,Балтика 9,34
3,Strongbow яблоко,10
4,Strongbow груша,2


---
### 4) Вывод информации о недостающих товарах на складе для существующих заказов для конкретного заказа (сложный)
```sql
create procedure ShowLackProductsByOrder_id
	@order_id int
as
select product_name, 
	abs(amount_on_stock - ordered) as lack_on_stock 
from
	(select product_name, 
		sum(OrdersOrderDetailsCustomersView.amount) as ordered, 
		stock.amount as amount_on_stock
	from OrdersOrderDetailsCustomersView 
		join stock on OrdersOrderDetailsCustomersView.product_id=stock.product_id
	where order_status != 'completed' and order_id = @order_id
group by product_name, stock.amount) 
as a
where (amount_on_stock - ordered) < 0
```

In [7]:
tableResult = pd.read_sql("""exec ShowLackProductsByOrder_id ?""", engine, params=['5'])
tableResult

,product_name,lack_on_stock
0,Балтика 7,5
1,Балтика 9,21
2,Strongbow яблоко,6


---
### 5) Вывод информации о недостающих товарах на складе для существующих заказов для конкретного заказчика (сложный)
```sql
create procedure ShowLackProductsByCustomer_name
	@customer_name varchar(30)
as
select product_name, 
	abs(amount_on_stock - ordered) as lack_on_stock 
from
	(select product_name, 
		sum(OrdersOrderDetailsCustomersView.amount) as ordered, 
		stock.amount as amount_on_stock
	from OrdersOrderDetailsCustomersView join stock on OrdersOrderDetailsCustomersView.product_id=stock.product_id
	where order_status != 'completed' and customer_name = @customer_name
	group by product_name, stock.amount) 
as a
where (amount_on_stock - ordered) < 0
```

In [8]:
tableResult = pd.read_sql("""exec ShowLackProductsByCustomer_name ?""", engine, params=['РосАл'])
tableResult

,product_name,lack_on_stock
0,Балтика 7,10
1,Балтика 9,27
2,Strongbow яблоко,6
3,Strongbow груша,2


---
### 6) Вывод информации обо всех заказах(сложный)
```sql
create procedure ShowAllOrders
as
select order_id, 
	   customer_name, 
	   order_date,
	   order_status,
	   total_prime_cost,
	   total_cost 
from Orders 
	join customers 
		on orders.customer_id=customers.customer_id
```

In [9]:
tableResult = pd.read_sql("""exec ShowAllOrders""", engine)
tableResult

,order_id,customer_name,order_date,order_status,total_prime_cost,total_cost
0,1,Красное&Белое,2022-05-23 14:25:10,pending,479.0,575.61
1,2,Красное&Белое,2022-05-24 14:25:10,processing,571.0,730.86
2,3,РосАл,2022-05-25 14:25:10,pending,1155.0,1265.34
3,4,РосАл,2022-05-26 14:25:10,completed,868.0,1121.68
4,5,РосАл,2022-05-27 14:25:10,cancelled,2628.0,3114.18


---
### 7) Вывод информации о своих заказах для заказчика
```sql
create procedure ShowOrdersByCustomer_id
	@customer_id int
as
select order_id,
	order_date,
	order_status, 
	total_cost 
from Orders
where customer_id = @customer_id
```

In [10]:
##id
tableResult = pd.read_sql("""exec ShowOrdersByCustomer_id ?""", engine, params={'2'})
tableResult

,order_id,order_date,order_status,total_cost
0,3,2022-05-25 14:25:10,pending,1265.34
1,4,2022-05-26 14:25:10,completed,1121.68
2,5,2022-05-27 14:25:10,cancelled,3114.18


---
### 8) Вывод информации о своих заказах для заказчика по статусу заказа
```sql
create procedure ShowOrdersByCustomer_idAndOrder_status
	@customer_id int,
	@order_status varchar(30)
as
select order_id,
	order_date,
	order_status, 
	total_cost
from Orders
where customer_id = @customer_id and order_status = @order_status
```

In [11]:
tableResult = pd.read_sql("""exec ShowOrdersByCustomer_idAndOrder_status ?, ?""", engine, params=['2', 'cancelled'])
tableResult

,order_id,order_date,order_status,total_cost
0,5,2022-05-27 14:25:10,cancelled,3114.18


---
### 9) Вывод содержимого заказа(сложный)
```sql
create procedure ShowOrderDetails
@order_id int
as
select product_name, amount, price, cost
	from OrdersOrderDetailsCustomersView
	where order_id = @order_id
```

In [12]:
tableResult = pd.read_sql("""exec ShowOrderDetails ?""", engine, params=['2'])
tableResult

,product_name,amount,price,cost
0,Балтика 8,2,93.84,187.68
1,Балтика 9,3,76.18,228.54
2,Strongbow яблоко,4,78.66,314.64


---
### 10) Вывод информации о заказчиках
```sql
create procedure ShowCustomers
as
select customer_name,
	email,
	phone_number,
	address
from customers
```

In [13]:
tableResult = pd.read_sql("""exec ShowCustomers""", engine)
tableResult

,customer_name,email,phone_number,address
0,Красное&Белое,red_white@mail.ru,88005553535,"ул. Пушкина, д.10"
1,РосАл,rosal@mail.ru,88005553536,"ул. Пушкина, д.11"


---
### 11) Вывод значения прибыли завода за все время
```sql
create procedure ShowAllIncome
as
select sum(total_cost - total_prime_cost) as income 
	from Orders
where order_status='completed'
```

In [14]:
tableResult = pd.read_sql("""exec ShowAllIncome""", engine)
tableResult

,income
0,253.68


---
### 12) Вывод значения прибыли завода от конкретного заказчика
```sql
create procedure ShowIncomeByCustomer_id
	@customer_id int
as
select sum(total_cost - total_prime_cost) as income 
	from Orders
where order_status='completed' and customer_id=@customer_id
```

In [15]:
tableResult = pd.read_sql("""exec ShowIncomeByCustomer_id ?""", engine, params=['2'])
tableResult

,income
0,253.68


---
### 13) Вывод информации о сотрудниках завода
```sql
create procedure ShowWorkers
as
select email, 
	username, 
	first_name, 
	second_name 
from workers 
	join users on workers.user_id=users.user_id
```

In [16]:
tableResult = pd.read_sql("""exec ShowWorkers""", engine)
tableResult

,email,username,first_name,second_name
0,v.pupkin@brewery.com,v.pupkin,Василий,Пупкин
1,n.pechkin@brewery.com,n.pechkin,Николай,Печкин


## Вставка

---
### 1) Вставка товара в базу товара
```sql
create procedure InsertProduct
	@product_name varchar(30),
	@product_type varchar(30),
	@upc_code varchar(12),
	@prime_price decimal(8, 2)
as
INSERT INTO products(product_name, product_type, upc_code, prime_price) 
	VALUES (@product_name, @product_type, @upc_code, @prime_price)
GO
```

In [17]:
with engine.begin() as conn:
	conn.execute("exec InsertProduct ?, ?, ?, ?", 'Балтика 0', 'Пиво', '123456789126', 60)
tableResult = pd.read_sql("exec ShowProductsBase", engine)
tableResult

,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 7,Пиво,123456789121,67.0,92.46,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0


---
### 2) Вставка учетной записи в таблицу сотрудников завода
```sql
create procedure InsertWorker
	@email varchar(30),
	@first_name varchar(30),
	@second_name varchar(30),
	@user_password varchar(30)
as
BEGIN
	INSERT INTO users(username,
			user_password,
			user_role) 
		VALUES (left(@email, charindex('@', @email) - 1),
				HASHBYTES('SHA2_256', @user_password), 
				'worker');
	INSERT INTO workers(user_id, 
			email,
			first_name, 
			second_name)
		VALUES ((select user_id from users
				where username=left(@email,  charindex('@', @email) - 1)),
			@email,
			@first_name,
			@second_name
		)
END
GO
```

In [27]:
with engine.begin() as conn:
	conn.execute("exec InsertWorker ?, ?, ?, ?", 'aboba@brewery.com', 'Aboba', 'Abobov', 'aboba12345')
	conn.execute("exec InsertWorker ?, ?, ?, ?", 'aboba@brewery.com', 'Aboba', 'Abobov', 'aboba12345')
tableResult = pd.read_sql("select * from users", engine)
display(tableResult)
tableResult = pd.read_sql("select * from workers", engine)
display(tableResult)

DBAPIError: (pyodbc.Error) ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]Общая ошибка сети. Обратитесь к документации по сети. (11)')
[SQL: exec InsertWorker ?, ?, ?, ?]
[parameters: ('aboba@brewery.com', 'Aboba', 'Abobov', 'aboba12345')]
(Background on this error at: https://sqlalche.me/e/14/dbapi)

---
### 3) Вставка учетной записи в таблицу заказчиков
```sql
create procedure InsertCustomer
	@customer_name varchar(30),
	@email varchar(30),
	@phone_number varchar(30),
	@address varchar(30),
	@user_password varchar(30)
as
BEGIN
	INSERT INTO users(username,
			user_password,
			user_role) 
		VALUES (left(@email, charindex('@', @email) - 1),
				HASHBYTES('SHA2_256', @user_password), 
				'customer');
	INSERT INTO customers(user_id, 
			email,
			customer_name, 
			phone_number,
			address)
		VALUES ((select user_id from users
				where username=left(@email,  charindex('@', @email) - 1)),
			@email,
			@customer_name,
			@phone_number,
			@address
		)
END
GO
```

In [19]:
with engine.begin() as conn:
	conn.execute("exec InsertCustomer ?, ?, ?, ?, ?", 'Alcomarket', 'alcomarket@mail.ru', '89775993055', 'Abobovaya 37', 'password12345')
tableResult = pd.read_sql("select * from users", engine)
display(tableResult)
tableResult = pd.read_sql("select * from customers", engine)
display(tableResult)

,user_id,username,user_password,user_role
0,1,red_white,b'2|\xfc\x94\x0bu\xdb\x05p\xcc&{ \xd6\xebv\xa5...,customer
1,2,rosal,b'n\x9f`\xf3\x12K/\x99\xbe9A\x10\x0f\xa1-P\x10...,customer
2,3,v.pupkin,b'\xdf\xb6\t\xcc\xd2\xf13\x9b\xdb\xd7G\xef\xf8...,worker
3,4,n.pechkin,b'\x99\xbc\x98\xf3\x96P\xfbji\xc9\\\x14\xd6\x8...,worker
4,5,aboba,b'r\x88&\xcc}\xa5\xf5\x13(\xaddQ\xdd\xae\xb2\x...,worker
5,6,alcomarket,b'7\x00\xad\xf1\xf2_\xab\x82\x02\xc14<K\x0bN?\...,customer


,user_id,customer_id,email,customer_name,phone_number,address
0,1,1,red_white@mail.ru,Красное&Белое,88005553535,"ул. Пушкина, д.10"
1,2,2,rosal@mail.ru,РосАл,88005553536,"ул. Пушкина, д.11"
2,6,3,alcomarket@mail.ru,Alcomarket,89775993055,Abobovaya 37


---
### 4) Вставка нового заказа
```sql
create procedure InsertOrder
	@customer_id INT,
	@order_date DATETIME
as
	INSERT INTO orders(customer_id, order_date) 
		VALUES (@customer_id, @order_date)
```
---

```sql
create procedure InsertOrder_details
	@order_id INT,
	@product_id INT,
	@amount INT
as
	INSERT INTO order_details(order_id, product_id, amount) 
		VALUES (@order_id, @product_id, @amount)
```


In [20]:
with engine.begin() as conn:
	conn.execute("exec InsertOrder ?, ?", 1, '2022-05-23T14:25:10')
	conn.execute("exec InsertOrder_details ?, ?, ?", 6, 1, 5)
	conn.execute("exec InsertOrder_details ?, ?, ?", 6, 2, 5)
tableResult = pd.read_sql("select * from orders", engine)
display(tableResult)
tableResult = pd.read_sql("exec ShowAllOrders", engine)
display(tableResult)


,order_id,customer_id,order_date,order_status,total_prime_cost,total_cost
0,1,1,2022-05-23 14:25:10,pending,479.0,575.61
1,2,1,2022-05-24 14:25:10,processing,571.0,730.86
2,3,2,2022-05-25 14:25:10,pending,1155.0,1265.34
3,4,2,2022-05-26 14:25:10,completed,868.0,1121.68
4,5,2,2022-05-27 14:25:10,cancelled,2628.0,3114.18
5,6,1,2022-05-23 14:25:10,pending,675.0,885.25


,order_id,customer_name,order_date,order_status,total_prime_cost,total_cost
0,1,Красное&Белое,2022-05-23 14:25:10,pending,479.0,575.61
1,2,Красное&Белое,2022-05-24 14:25:10,processing,571.0,730.86
2,3,РосАл,2022-05-25 14:25:10,pending,1155.0,1265.34
3,4,РосАл,2022-05-26 14:25:10,completed,868.0,1121.68
4,5,РосАл,2022-05-27 14:25:10,cancelled,2628.0,3114.18
5,6,Красное&Белое,2022-05-23 14:25:10,pending,675.0,885.25


## Изменение

---
### 1) Изменение информации о товаре в базе
```sql
create procedure UpdateProduct
	@product_id int,
	@product_name varchar(30),
	@product_type varchar(30),
	@upc_code varchar(30),
	@prime_price varchar(30)
as
	UPDATE products
		set product_name = @product_name,
			product_type = @product_type,
			upc_code = @upc_code,
			prime_price = @prime_price
	where product_id=@product_id
```


In [21]:
tableResult = pd.read_sql("exec ShowProductsBase", engine)
display(tableResult)
with engine.begin() as conn:
	conn.execute("exec UpdateProduct ?, ?, ?, ?, ?", 1, 'Балтика 77', 'Пиво', '123456789129', 60)
tableResult = pd.read_sql("exec ShowProductsBase", engine)
display(tableResult)

,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 7,Пиво,123456789121,67.0,92.46,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0


,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 77,Пиво,123456789129,60.0,82.80,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0


---
### 2) Изменение скидки на товар
```sql
create procedure UpdateDiscount
	@product_id int,
	@discount decimal(3, 2)
as
	UPDATE discounts
		set discount = @discount
	where product_id=@product_id
```


In [22]:
tableResult = pd.read_sql("exec ShowProductsBase", engine)
display(tableResult)
with engine.begin() as conn:
	conn.execute("exec UpdateDiscount ?, ?", 1, 0.18)
tableResult = pd.read_sql("exec ShowProductsBase", engine)
display(tableResult)

,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 77,Пиво,123456789129,60.0,82.80,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0


,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 77,Пиво,123456789129,60.0,82.80,67.90,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0


---
### 3) Изменение количества товара на складе
```sql
create procedure UpdateAmountOnStock
	@product_id int,
	@amount int
as
	UPDATE stock
		set amount = @amount
	where product_id=@product_id
```

In [23]:
tableResult = pd.read_sql("exec ShowProductsBase", engine)
display(tableResult)
with engine.begin() as conn:
	conn.execute("exec UpdateAmountOnStock ?, ?", 1, 10)
tableResult = pd.read_sql("exec ShowProductsBase", engine)
display(tableResult)

,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 77,Пиво,123456789129,60.0,82.80,67.90,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0


,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 77,Пиво,123456789129,60.0,82.80,67.90,10
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0


---
### 4) Изменение данных о учетной записи заказчика
```sql
create procedure UpdateCustomer
	@customer_id int,
	@phone_number varchar(30),
	@address varchar(30)
as
	UPDATE customers
		set phone_number = @phone_number,
			address = @address
	where customer_id=@customer_id
```

In [24]:
tableResult = pd.read_sql("exec ShowCustomers", engine)
display(tableResult)
with engine.begin() as conn:
	conn.execute("exec UpdateCustomer ?, ?, ?", 1, '12345678911', 'updatedStreet 13')
tableResult = pd.read_sql("exec ShowCustomers", engine)
display(tableResult)

,customer_name,email,phone_number,address
0,Красное&Белое,red_white@mail.ru,88005553535,"ул. Пушкина, д.10"
1,РосАл,rosal@mail.ru,88005553536,"ул. Пушкина, д.11"
2,Alcomarket,alcomarket@mail.ru,89775993055,Abobovaya 37


,customer_name,email,phone_number,address
0,Красное&Белое,red_white@mail.ru,12345678911,updatedStreet 13
1,РосАл,rosal@mail.ru,88005553536,"ул. Пушкина, д.11"
2,Alcomarket,alcomarket@mail.ru,89775993055,Abobovaya 37


---
### 5) Изменение статуса товара
```sql
create procedure UpdateOrderStatus
	@order_id int,
	@order_status varchar(30)
as
	UPDATE orders
		set order_status = @order_status
	where order_id=@order_id
```

In [25]:
tableResult = pd.read_sql("exec ShowAllOrders", engine)
display(tableResult)
with engine.begin() as conn:
	conn.execute("exec UpdateOrderStatus ?, ?", 1, 'cancelled')
tableResult = pd.read_sql("exec ShowAllOrders", engine)
display(tableResult)

,order_id,customer_name,order_date,order_status,total_prime_cost,total_cost
0,1,Красное&Белое,2022-05-23 14:25:10,pending,479.0,575.61
1,2,Красное&Белое,2022-05-24 14:25:10,processing,571.0,730.86
2,3,РосАл,2022-05-25 14:25:10,pending,1155.0,1265.34
3,4,РосАл,2022-05-26 14:25:10,completed,868.0,1121.68
4,5,РосАл,2022-05-27 14:25:10,cancelled,2628.0,3114.18
5,6,Красное&Белое,2022-05-23 14:25:10,pending,675.0,885.25


,order_id,customer_name,order_date,order_status,total_prime_cost,total_cost
0,1,Красное&Белое,2022-05-23 14:25:10,cancelled,479.0,575.61
1,2,Красное&Белое,2022-05-24 14:25:10,processing,571.0,730.86
2,3,РосАл,2022-05-25 14:25:10,pending,1155.0,1265.34
3,4,РосАл,2022-05-26 14:25:10,completed,868.0,1121.68
4,5,РосАл,2022-05-27 14:25:10,cancelled,2628.0,3114.18
5,6,Красное&Белое,2022-05-23 14:25:10,pending,675.0,885.25


## Удаление

### Удаление учетной записи сотрудника завода
```sql
create procedure DeleteWorker
	@worker_id int
as
	DELETE from users
		where user_id=@worker_id
```

In [26]:
tableResult = pd.read_sql("exec ShowWorkers", engine)
display(tableResult)
tableResult = pd.read_sql("select * from users", engine)
display(tableResult)
with engine.begin() as conn:
	conn.execute("exec DeleteWorker ?", 5)
tableResult = pd.read_sql("exec ShowWorkers", engine)
display(tableResult)
tableResult = pd.read_sql("select * from users", engine)
display(tableResult)

,email,username,first_name,second_name
0,v.pupkin@brewery.com,v.pupkin,Василий,Пупкин
1,n.pechkin@brewery.com,n.pechkin,Николай,Печкин
2,aboba@brewery.com,aboba,Aboba,Abobov


,user_id,username,user_password,user_role
0,1,red_white,b'2|\xfc\x94\x0bu\xdb\x05p\xcc&{ \xd6\xebv\xa5...,customer
1,2,rosal,b'n\x9f`\xf3\x12K/\x99\xbe9A\x10\x0f\xa1-P\x10...,customer
2,3,v.pupkin,b'\xdf\xb6\t\xcc\xd2\xf13\x9b\xdb\xd7G\xef\xf8...,worker
3,4,n.pechkin,b'\x99\xbc\x98\xf3\x96P\xfbji\xc9\\\x14\xd6\x8...,worker
4,5,aboba,b'r\x88&\xcc}\xa5\xf5\x13(\xaddQ\xdd\xae\xb2\x...,worker
5,6,alcomarket,b'7\x00\xad\xf1\xf2_\xab\x82\x02\xc14<K\x0bN?\...,customer


,email,username,first_name,second_name
0,v.pupkin@brewery.com,v.pupkin,Василий,Пупкин
1,n.pechkin@brewery.com,n.pechkin,Николай,Печкин


,user_id,username,user_password,user_role
0,1,red_white,b'2|\xfc\x94\x0bu\xdb\x05p\xcc&{ \xd6\xebv\xa5...,customer
1,2,rosal,b'n\x9f`\xf3\x12K/\x99\xbe9A\x10\x0f\xa1-P\x10...,customer
2,3,v.pupkin,b'\xdf\xb6\t\xcc\xd2\xf13\x9b\xdb\xd7G\xef\xf8...,worker
3,4,n.pechkin,b'\x99\xbc\x98\xf3\x96P\xfbji\xc9\\\x14\xd6\x8...,worker
4,6,alcomarket,b'7\x00\xad\xf1\xf2_\xab\x82\x02\xc14<K\x0bN?\...,customer
